# Assignment 3

In [1]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import min, avg, max, stddev, corr
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. But we discurage using RDD at this point in time.

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. Everything can be implemented using SQL only if you like.

In [3]:
def minTemperature():
    return spark.table("washing").agg(min("temperature").alias("mintemp")).first().mintemp

Please now do the same for the mean of the temperature

In [22]:
def meanTemperature():
    return spark.table("washing").agg(avg("temperature").alias("meantemp")).first().meantemp

Please now do the same for the maximum of the temperature

In [15]:
def maxTemperature():
    return spark.table("washing").agg(max("temperature").alias("maxtemp")).first().maxtemp

Please now do the same for the standard deviation of the temperature

In [6]:
def sdTemperature():
    return spark.table("washing").agg(stddev("temperature").alias("sdtemp")).first().sdtemp

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four positions in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [7]:
def skewTemperature():    
    return spark.sql("""
SELECT 
    (
        1/COUNT(*)
    ) *
    SUM (
        POWER(temperature-%s,3)/POWER(%s,3)
    )

as sktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().sktemperature

Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [8]:
def kurtosisTemperature():    
        return spark.sql("""
SELECT 
    (
        1/COUNT(*)
    ) *
    SUM (
        POWER(temperature-%s,4)/POWER(%s,4)
    )
as ktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().ktemperature


Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [9]:
def correlationTemperatureHardness():
    return spark.table("washing").select(corr("temperature", "hardness")).first()[0]

Now it is time to grab a PARQUET file and create a dataframe out of it. Using SparkSQL you can handle it like a database. 

In [10]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     NULL|      77| NULL|        100|1547808723923|   NULL|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    NULL|      NULL|     NULL|    NULL| 1046|       NULL|1547808729917|   NULL|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    NULL|      NULL|       71|    NULL| NULL|       NULL|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     NULL|      75| NULL|         86|1547808738999|   NULL|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    NULL|      NULL|       75|    NULL| NULL|   

Now let's test the functions you've implemented

In [11]:
min_temperature = 0
mean_temperature = 0
max_temperature = 0
sd_temperature = 0
skew_temperature = 0
kurtosis_temperature = 0
correlation_temperature = 0


In [12]:
min_temperature = minTemperature()
print(min_temperature)

80


In [23]:
mean_temperature = meanTemperature()
print(mean_temperature)

90.03800298062593


In [16]:
max_temperature = maxTemperature()
print(max_temperature)

100


In [17]:
sd_temperature = sdTemperature()
print(sd_temperature)

6.1007610586219725


In [18]:
skew_temperature = skewTemperature()
print(skew_temperature)

-6.024439095555274


In [19]:
kurtosis_temperature = kurtosisTemperature()
print(kurtosis_temperature)

16.172886955094025


In [20]:
correlation_temperature = correlationTemperatureHardness()
print(correlation_temperature)

0.017754069047296324


Congratulations, you are done, please submit this notebook to the grader.